### 03. Silver Transformations

#### Data Cleaning

In [1]:
today_date = '2024-09-17'
workspace = "fabric_DEV"

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 3, Finished, Available, Finished)

In [3]:
fabric_bronze_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/bronze_data"

from pyspark.sql.functions import col
df = spark.read.format('delta').load(fabric_bronze_path).filter(col('Processing_Date') == str(today_date))

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0f385b1d-9bc5-41d9-a7ee-eec05db66bf2)

In [ ]:
display(df)

#### 01. Handling Duplicates

In [4]:
print('Count of rows before deleting duplicates :' , df.count())

df_nodups = df.dropDuplicates()

print('Count of rows before deleting duplicates :' , df_nodups.count())

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 6, Finished, Available, Finished)

Count of rows before deleting duplicates : 8
Count of rows before deleting duplicates : 8


#### 02 - Handle missing or NULL values

##### 02. a . Drop rows with missing critical values

In [5]:
print('Count before dropping missing criticial data rows : ', df_nodups.count() )

df_dropped = df_nodups.dropna(subset=['Student_ID','Course_ID','Enrollment_Date'])

print('Count After dropping missing criticial data rows : ', df_dropped.count() )


StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 7, Finished, Available, Finished)

Count before dropping missing criticial data rows :  8
Count After dropping missing criticial data rows :  8


##### 02. b. Fill rows with default values

In [6]:
df_filled = df_dropped.fillna({
                "Age": 0,
                "Gender": "Unknown",
                "Status": "In-progress",
                "Final_Grade": "N/A",
                "Attendance_Rate": 0.0,
                "Time_Spent_on_Course_hrs": 0.0,
                "Assignments_Completed": 0,
                "Quizzes_Completed": 0,
                "Forum_Posts": 0,
                "Messages_Sent": 0,
                "Quiz_Average_Score": 0.0,
                "Assignment_Average_Score": 0.0,
                "Project_Score": 0.0,
                "Extra_Credit": 0.0,
                "Overall_Performance": 0.0,
                "Feedback_Score": 0.0,
                "Parent_Involvement": "Unknown",
                "Demographic_Group": "Unknown",
                "Internet_Access": "Unknown",
                "Learning_Disabilities": "Unknown",
                "Preferred_Learning_Style": "Unknown",
                "Language_Proficiency": "Unknown",
                "Participation_Rate": "Unknown",
                "Completion_Time_Days": 0,
                "Performance_Score": 0.0,
                "Course_Completion_Rate": 0.0,
                "Completion_Date": '12/31/9999'

            })

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 8, Finished, Available, Finished)

#### 03. Standardize Date Formats

In [8]:
from pyspark.sql.functions import to_date, col

df_format = df_filled.withColumn("Enrollment_Date", to_date(col("Enrollment_Date"), "M/d/yyyy"))\
         .withColumn("Completion_Date",to_date(col('Completion_Date'), "M/d/yyyy"))

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 10, Finished, Available, Finished)

In [9]:
display(df_format)

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5555ba7b-e4e6-4f92-8b0f-c94c47922c2d)

#### 04. Check for Logical consistency
##### Completion_Date > Enrollment_Date

In [11]:
df_consistent = df_format.filter(col("Completion_Date") >= col("Enrollment_Date"))

StatementMeta(, f322dcdc-c59c-4196-be79-81b3cb8da204, 13, Finished, Available, Finished)

### Business Transformations